In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay


In [ ]:
client = EfdClient('summit_efd')

In [ ]:
pdf = PdfPages("/home/craiglagegit/DATA/Shutter_State_Multiple_20Feb24.pdf")
startDay = 20240220
endDay = 20240220
fig = plt.figure(figsize=(15,5))
dayObs = startDay
while dayObs <= endDay:
    nextDayObs = calcNextDay(dayObs)
    dayString = dayObsIntToString(dayObs)
    nextDayString = dayObsIntToString(nextDayObs)
    
    try:
        start = Time(f"{dayString}T10:00:00", scale='utc')
        end = Time(f"{nextDayString}T10:00:00", scale='utc')
    
        state = await client.select_time_series('lsst.sal.ATDome.logevent_mainDoorState', \
                                            ['*'],  start, end)
        shut = await client.select_time_series('lsst.sal.ATDome.position', \
                                                    ['mainDoorOpeningPercentage'],  start, end)
        close_command = await client.select_time_series('lsst.sal.ATDome.command_closeShutter', \
                                                    ['*'],  start, end)
        open_command = await client.select_time_series('lsst.sal.ATDome.command_openShutter', \
                                                    ['*'],  start, end)
        summary_state = await client.select_time_series('lsst.sal.ATDome.logevent_summaryState', \
                                                    ['*'],  start, end)

        dome_move = await client.select_time_series('lsst.sal.ATDome.command_moveAzimuth', \
                                                    ['*'],  start, end)
        print(len(state), len(shut), len(close_command), len(summary_state), len(dome_move))
        if len(shut) == 0:
            dayObs = calcNextDay(dayObs)
            continue
        ax = fig.subplots(1,1)
        ax.set_title(f"Shutter_State_{dayObs}", fontsize=24)
        state['state'].plot(ax=ax, color='blue', label='Actual State')
        ax.set_yticks([1,2,3,4,5,6,7])
    
        ax.set_yticklabels(['Closed', 'Opened', 'Ajar', 'Opening', 'Closing','',''])
        ax.set_ylabel("Door State")
        ax1 = ax.twinx()
        shut['mainDoorOpeningPercentage'].plot(ax=ax1, color='red', label='Shutter Open Percentage')
        ax1.set_ylabel("Open percentage")
        ax1.set_ylim(-2,115)
    
        disabled = summary_state[summary_state['summaryState'] == 1]
        enabled = summary_state[summary_state['summaryState'] == 2]
        standby = summary_state[summary_state['summaryState'] == 5]
        fault = summary_state[summary_state['summaryState'] == 3]
    
        """
        disabled = Time("2024-02-12T12:00:00", scale='utc')
        enabled = Time("2024-02-12T18:44:00", scale='utc')
        ax1.axvline(disabled.isot, ls='--', color='cyan', label='DISABLED')
        ax1.axvline(enabled.isot, ls='--', color='orange', label='ENABLED')
        """
        for close in close_command.index[0:1]:
            ax1.axvline(close, ls='-.', color='green', label='Close Command')
        for open in open_command.index[0:1]:
            ax1.axvline(open, ls='-.', color='magenta', label='Open Command')
        for disable in disabled.index[0:1]:
            ax1.axvline(disable, ls='--', color='cyan', label='Disabled')
        for enable in enabled.index[0:1]:
            ax1.axvline(enable, ls='--', color='darkgreen', label='Enabled')
        for std in standby.index[0:1]:
            ax1.axvline(std, ls='--', color='orange', label='Standby')
        for flt in fault.index[0:1]:
            ax1.axvline(flt, ls='--', color='red', label='Fault')
        for move in dome_move.index[0:1]:
            ax1.axvline(move, ymin=0, ymax=0.1, ls='--', color='black', label='Dome_Move')
        """    
        for i, this_enable in enumerate(enables):
            for en in this_enable.index[0:1]:
                print(cscs[i], en)
                ax1.axvline(en, ls='--',  label=cscs[i])
        
        for i, this_enable in enumerate(enables):
            for en in this_enable.index:
                print(cscs[i], en)
                ax1.axvline(en, ls='--',  label='')
        """        
    
        for close in close_command.index:
            ax1.axvline(close, ls='-.', color='green', label='')
        for open in open_command.index:
            ax1.axvline(open, ls='-.', color='magenta', label='')
        for disable in disabled.index:
            ax1.axvline(disable, ls='--', color='cyan', label='')
        for enable in enabled.index:
            ax1.axvline(enable, ls='--', color='darkgreen', label='')
        for std in standby.index:
            ax1.axvline(std, ls='--', color='orange', label='')
        for flt in fault.index:
            ax1.axvline(flt, ls='--', color='red', label='')
        for move in dome_move.index:
            ax1.axvline(move, ymin=0, ymax=0.1, ls='--', color='black', label='')
    
    
        ax.legend(loc='lower left')
        ax1.legend(loc='upper left')
    
        
        pdf.savefig(fig)  # saves the current figure into a pdf page
        plt.clf()
        dayObs = calcNextDay(dayObs)
    except:
        dayObs = calcNextDay(dayObs)
        continue
pdf.close()


In [ ]:
close_command.index[0]

In [ ]:
for i, move in enumerate(dome_move.index):
    print(move, dome_move['azimuth'][i])

In [ ]:
dome_move['azimuth'].plot()

In [ ]:
len(open_command)

In [ ]:
end

In [ ]:
enables = []
cscs = ['ATDome', 'ATDomeTrajectory', 'ATMCS', 'ATPtg', 'ATAOS', 'ATHexapod', 'ATPneumatics']

for csc in cscs:
    query = f'lsst.sal.{csc}.logevent_summaryState'
    these_states = await client.select_time_series(query, ['*'],  start, end)
    enable = these_states[these_states['summaryState'] == 2]
    print(csc, len(enable))
    enables.append(enable)



In [ ]:
heartbeat = await client.select_time_series('lsst.sal.ATDome.logevent_heartbeat', \
                                            ['*'],  start, end)

In [ ]:
len(heartbeat)

In [ ]:
times = heartbeat['private_efdStamp'].values
times = times - times[0]
deltas = []
for i,time in enumerate(times):
    if i == 0:
        delta = times[1] - times[0]
        deltas.append(delta)
    else:
        delta = times[i] - times[i-1]
        deltas.append(delta)
        

In [ ]:
plt.plot(times, deltas)

In [ ]:
startPlot = Time("2024-02-06T01:14:00", scale='utc')
endPlot = Time("2024-02-06T01:35:00", scale='utc')
startPlot2 = startPlot.unix_tai - start.unix_tai
endPlot2 = endPlot.unix_tai - start.unix_tai

fig = plt.figure(figsize=(5,8))
plt.subplots_adjust(hspace = 0.5)
axs = fig.subplots(2,1)
axs[0].set_title(f"Shutter_State_{dayObs}", fontsize=16)
state['state'].plot(ax=axs[0], color='blue', label='Actual State')
axs[0].set_yticks([1,2,3,4,5,6,7])

axs[0].set_yticklabels(['Closed', 'Opened', 'Ajar', 'Opening', 'Closing','',''])
axs[0].set_ylabel("Door State")
axs[0].set_xlim(startPlot.isot, endPlot.isot)
axs[1].set_title("Time between heartbeats", fontsize=16)
axs[1].plot(times, deltas)
axs[1].set_xlim(startPlot2, endPlot2)
axs[1].set_ylabel("Time between successive\n heartbeats (sec)")
axs[1].set_xlabel("Time since plot start (sec)")
for time in state['private_kafkaStamp']:
    axs[1].axvline(time - start.unix_tai, ls = '-.', color='k')
for time in state['private_kafkaStamp']:
    axs[0].axvline(Time(time, format='unix_tai').utc.isot, ls = '-.', color='k')
plt.savefig("/home/craiglagegit/DATA/ATDome_Heartbeats_06Feb24.png")

In [ ]:
startPlot = Time("2024-02-06T01:14:00", scale='utc')
endPlot = Time("2024-02-06T01:35:00", scale='utc')

test = await client.select_time_series('lsst.sal.ATDome.logevent_doorEncoderExtremes', \
                                            ['*'],  startPlot, endPlot)
#test = await client.select_time_series('lsst.sal.ATDome.command_moveShutterMainDoor', \
#                                            ['*'],  start, end)
print(len(test))

In [ ]:
test.columns

In [ ]:
startPlot2 = startPlot.unix_tai - start.unix_tai
endPlot2 = endPlot.unix_tai - start.unix_tai

fig = plt.figure(figsize=(5,8))
plt.subplots_adjust(hspace = 0.5)
axs = fig.subplots(2,1)
axs[0].set_title(f"Shutter_State_{dayObs}", fontsize=16)
state['state'].plot(ax=axs[0], color='blue', label='Actual State')
axs[0].set_yticks([1,2,3,4,5,6,7])

axs[0].set_yticklabels(['Closed', 'Opened', 'Ajar', 'Opening', 'Closing','',''])
axs[0].set_ylabel("Door State")
axs[0].set_xlim(startPlot.isot, endPlot.isot)







axs[1].set_title("Test", fontsize=16)
test['mainClosed'].plot(ax=axs[1], color='red', label='mainClosed')
test['mainOpened'].plot(ax=axs[1], color='red', label='mainOpened')
#axs[1].plot(times, deltas)
axs[1].set_xlim(startPlot.isot, endPlot.isot)
#axs[1].set_ylabel("Time between successive\n heartbeats (sec)")
#axs[1].set_xlabel("Time since plot start (sec)")
for time in state['private_kafkaStamp']:
    axs[1].axvline(Time(time, format='unix_tai').utc.isot, ls = '-.', color='k')
for time in test['private_kafkaStamp']:
    axs[0].axvline(Time(time, format='unix_tai').utc.isot, ls = '-.', color='k')
#plt.savefig("/home/craiglagegit/DATA/ATDome_Heartbeats_06Feb24.png")

In [ ]:
summary_state = await client.select_time_series('lsst.sal.ATDome.logevent_summaryState', \
                                                    ['*'],  start, end)

In [ ]:
summary_state.columns

In [ ]:
disabled = summary_state[summary_state['summaryState'] == 1]
enabled = summary_state[summary_state['summaryState'] == 2]
standby = summary_state[summary_state['summaryState'] == 5]
fault = summary_state[summary_state['summaryState'] == 3]

In [ ]:
disabled